In [1]:
import pandas as pd
import numpy as np
import sys
import pickle

sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import evaluate

import xgboost as xgb
from sklearn.linear_model import LinearRegression,LogisticRegression

(CVXPY) Jul 05 11:32:59 AM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 05 11:32:59 AM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 05 11:32:59 AM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/e

/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Data Module

In [2]:
dataset = CoronnaCERTAINDataset(
    library_root='/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
    challenge="binary_classification", #option: regression, regression_delta, classification, binary_classification
    dataset='CORRONA CERTAIN', 
    process_approach='KVB', #option: KVB, SC
    imputation='KNN', #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group='bionaive TNF', #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points=(0,3), 
    train_test_rate=0.8,
    save_csv=False, 
    random_state=2022)

Label Encoder, 0: Non-responders (No Response), 1: Responders(Good, Moderate)


In [3]:
# read train, test from dataloader
train, train_loc = dataset.get_train()
test, test_loc = dataset.get_test()

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 120 to 37
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   grp                  214 non-null    float64
 1   Type I IFN activity  214 non-null    float64
 2   IFNβ activity final  214 non-null    float64
 3   IFNα activity final  214 non-null    float64
 4   IFNβ/α ratio final   214 non-null    float64
 5   age                  214 non-null    float64
 6   gender               214 non-null    float64
 7   final_education      214 non-null    float64
 8   race_grp             214 non-null    float64
 9   ethnicity            214 non-null    float64
 10  weight               214 non-null    float64
 11  height               214 non-null    float64
 12  newsmoker            214 non-null    float64
 13  drinker              214 non-null    float64
 14  drinksperwk          214 non-null    float64
 15  duration_ra          214 non-null    fl

In [5]:
# get x,y train and test set
X_train = train.iloc[:,:-1] # train.iloc[:,-2]
y_train = train.iloc[:,-1] 
X_test = test.iloc[:,:-1] # test.iloc[:,-2]
y_test = test.iloc[:,-1]

In [8]:
y_train

120   NaN
113   NaN
251   NaN
156   NaN
167   NaN
       ..
253   NaN
254   NaN
61    NaN
21    NaN
37    NaN
Name: DrugResponse_binary, Length: 214, dtype: float64

In [6]:
# initialize evaluation module
aml = evaluate.AutoBuild(seed=1, project_name="EHR_RA_SC", challenge=dataset.challenge)

# Model Module

In [7]:
# modelmodule
if "regression" in dataset.challenge:
    model_id = 'xgb'
    save_path = f'../Models/{dataset.challenge}_{model_id}.pkl'
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    # optional, to save and load models using pickle
    pickle.dump(model, open(save_path, 'wb'))
    model = pickle.load(open(save_path, 'rb'))
    
    aml.evaluate(model_id, model, test)
elif "classification" in dataset.challenge:
    model = xgb.XGBClassifier(use_label_encoder=False)
    model.fit(X_train, y_train)
    aml.evaluate("xgb", model, test)

ValueError: The label must consist of integer labels of form 0, 1, 2, ..., [num_class - 1].

In [ ]:
# linear model
if "regression" in dataset.challenge:
    model = LinearRegression()
    model.fit(X_train, y_train)
    aml.evaluate("lr", model, test) # input: model and test
elif "classification" in dataset.challenge:
    model = LogisticRegression()
    model.fit(X_train, y_train)
    aml.evaluate("lr", model, test) # input: model and test

In [ ]:
X_test

# Evaluation on test set

In [ ]:
regression, classification = aml.leaderboard()

In [ ]:
regression

In [ ]:
classification

In [ ]:
# rows: true, columns: pred
aml.confusion_matrix("xgb",plot=True, normalize=True)

In [ ]:
# true
aml.saved_model['xgb'][0].value_counts()

In [ ]:
# pred
aml.saved_model['xgb'][1].value_counts()

In [ ]:
aml.plot_results('regression','MSE')

In [ ]:
aml.plot_results('regression','Pearson_Correlation')

In [ ]:
aml.plot_results('classification','Accuracy')

In [ ]:
aml.save_outputs(dataset,"../leaderboard")